In [1]:
from openai import OpenAI
import json

In [ ]:
# load encounter events
with open("C:/Users/Kaden/OneDrive/UCB/Research/IntellikGraph/Code/data/synthetic_events.json") as file:
    data = json.load(file)
data[0]

{'EncounterID': 'Encounter_1',
 'DayUTC': '2025-06-10',
 'TimeBlockUTC': '1200-1500Z',
 'Zone': 'EEZ:Philippines',
 'VesselA': {'Name': 'PH_Trawler_302',
  'Type': 'SmallTrawler',
  'Flag': 'PH',
  'LastPort': 'Offshore_Cebu',
  'RecentBehavior': 'LoiteringPrev24h',
  'WeatherOK': True,
  'OSINT48h': True},
 'VesselB': {'Name': 'ID_Reefer_417',
  'Type': 'ReeferCarrier',
  'Flag': 'ID',
  'LastPort': 'Cebu_Port',
  'RecentBehavior': 'PortDeparture20hAgo',
  'WeatherOK': True,
  'OSINT48h': True},
 'DerivedMetrics': {'MinSep_nm': 0.8, 'Duration_min': 22, 'CrossFlag': True},
 'AttentionRequired': True,
 'ExplanationTrigger': 'Cross-flag trawler-reefer meeting in same EEZ soon after loitering and port departure; corroborated by OSINT and calm seas.'}

### Use LLM to generate explanation of event

In [3]:
client = OpenAI()

In [21]:
SYSTEM_PROMPT = """
    You are a maritime analyst. 
    Given JSON data describing an encounter between two vessels, write a short, factual summary (3-5 sentences) explaining what occurred.
    Use only the information provided in the JSON.
    Do not invent details or numbers.
    Keep the explanation concise, professional, and neutral in tone. It should read like smoothly like a story.
"""

In [22]:
res = client.beta.chat.completions.parse(
    model = "gpt-5",
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Data: {str(data[0])}"}
    ],
)
explanation = res.choices[0].message.content

In [23]:
import textwrap
print(textwrap.fill(explanation, width = 120))

On 2025-06-10 between 1200-1500Z in the Philippines EEZ, the small trawler PH_Trawler_302 (PH flag) and the reefer
carrier ID_Reefer_417 (ID flag) were in proximity. They closed to a minimum separation of 0.8 nm and remained near each
other for about 22 minutes. The trawler had been loitering in the prior 24 hours near Offshore_Cebu, while the reefer
had departed Cebu_Port around 20 hours earlier. With calm conditions and recent OSINT corroboration, this cross-flag
trawler–reefer meeting drew attention.


### Retrieve relevant subgraph from knowledge graph